In [1]:
library(forecast)
library(M4comp2018)
library(dplyr)

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
etsl=load('Hourly_ets_datalist.RData')
ets_datalist <- eval(parse(text = etsl))
thetafl=load('Hourly_thetaf_datalist.RData')
thetaf_datalist <- eval(parse(text = thetafl))

In [3]:
ets_predh=ets_datalist[[2]]
thetaf_predh=thetaf_datalist[[2]]

In [4]:
ets_pred_res=ets_datalist[[1]]
thetaf_pred_res=thetaf_datalist[[1]]

In [5]:
etsoptl=load('Hourly_ets_opt_pre_result1.RData')
ets_opt_pre_list<- eval(parse(text = etsoptl))
thetafoptl=load('Hourly_thetaf_opt_pre_result1.RData')
thetaf_opt_pre_list<- eval(parse(text = thetafoptl))

In [6]:
time_matrix <- matrix(0,ncol = 3, nrow =3)
colnames(time_matrix) <- c("user_time", "system_time", "elapsed_time")
start_time = Sys.time()

In [7]:
ets_pr_min=ets_opt_pre_list[[1]]
ets_pr_mean=ets_opt_pre_list[[2]]
thetaf_pr_min=thetaf_opt_pre_list[[1]]
thetaf_pr_mean=thetaf_opt_pre_list[[2]]

In [8]:
head(ets_pr_min)

xgbclsmin,xgbregmin,lgbclsmin,lgbregmin
3,2.9910419,0,1.380126
1,0.9993294,4,1.044559
1,1.0033541,4,1.167730
2,2.0007987,0,2.604530
4,3.9986844,4,3.450399
4,4.0005698,4,3.868265


In [9]:
ets_pr_min=ets_pr_min+1
ets_pr_mean=ets_pr_mean+1
thetaf_pr_min=thetaf_pr_min+1
thetaf_pr_mean=thetaf_pr_mean+1


In [10]:
head(ets_pr_min)

xgbclsmin,xgbregmin,lgbclsmin,lgbregmin
4,3.991042,1,2.380126
2,1.999329,5,2.044559
2,2.003354,5,2.167730
3,3.000799,1,3.604530
5,4.998684,5,4.450399
5,5.000570,5,4.868265


In [11]:
round5=function(x)
    {
    l=round(x)
    for(i in 1:length(l))
    {
    if(l[i]>5)
        {
        l[i]=5
    }
    if(l[i]<1)
    {
        l[i]=1
    }
}
    return(l)
}

In [12]:
ets_pr_min[,2]=round5(ets_pr_min[,2])
ets_pr_min[,4]=round5(ets_pr_min[,4])

In [13]:
ets_pr_mean[,2]=round5(ets_pr_mean[,2])
ets_pr_mean[,4]=round5(ets_pr_mean[,4])

In [14]:
thetaf_pr_min[,2]=round5(thetaf_pr_min[,2])
thetaf_pr_min[,4]=round5(thetaf_pr_min[,4])

In [15]:
thetaf_pr_mean[,2]=round5(thetaf_pr_mean[,2])
thetaf_pr_mean[,4]=round5(thetaf_pr_mean[,4])

In [16]:
head(ets_pr_min)

xgbclsmin,xgbregmin,lgbclsmin,lgbregmin
4,4,1,2
2,2,5,2
2,2,5,2
3,3,1,4
5,5,5,4
5,5,5,5


In [17]:
Hourly_M4 <- Filter(function(l) l$period == "Hourly", M4)

In [18]:
data=Hourly_M4

In [19]:
datalength=length(data)
h=data[[1]]$h
m=5
n=4
freq=frequency(data[[1]]$x)

In [20]:
realxx=matrix(0,datalength,h)
for(i in 1:datalength){
    realxx[i,]=data[[i]]$xx
}

In [21]:
length5=matrix(0,datalength,m)
for(k in 1:datalength)
{
y = data[[k]]$x
y_l=length(y)
loc = 1:length(y)
loc_m = as.integer(loc*m/length(y))
filt_d = data.frame(y,loc_m)
filt_0=filter(filt_d ,loc_m==0)
m_l=count(filt_0)
m_l=as.integer(m_l)
length5[k,]=c(0,1*m_l,2*m_l,3*m_l,4*m_l)
    }

In [22]:
end_time = Sys.time()
time_matrix[1,]=end_time-start_time

In [23]:
end_time-start_time

Time difference of 1.072885 mins

## ETS

In [24]:
start_time = Sys.time()

In [25]:
set.seed(100)
index = sample(2,length(data),replace = TRUE,prob=c(0.7,0.3))
testindex=c(1:length(data))[index==2]

In [26]:
meanpre=function(pred_array,opt_pr,h,testindex)
    {
    data_length=length(testindex)
    meanpreh=matrix(0,data_length,h)
    for (i in 1:data_length){
    index=testindex[i]
    loc=opt_pr[index]
    predh=pred_array[index,loc,,]
    predh=unique(predh)
    meanpreh[i,]=apply(predh,2,mean)
}
    return(meanpreh)
}

In [27]:
predres=function(pred_matrix,data,opt_pr,lengthmatrix,h,testindex)
    {
    data_length=length(testindex)
    pred_res=matrix(0,data_length,6)
    model=ets(data[[1]]$x)
    fore_l=forecast(model,h=h)
    freq=frequency(data[[1]]$x)
    for (i in 1:data_length){
    index=testindex[i]
    loc=opt_pr[index]
    start=lengthmatrix[i,loc]
    y_all=data[[index]]$x
    y=y_all[start:length(y_all)]
    fore_l$x=ts(y,frequency=freq,end=end(y_all))
    real=data[[index]]$xx
    fore_l$mean=ts(pred_matrix[i,],start=start(real),frequency=freq)
    res=accuracy(fore_l,real)
    pred_res[i,]=res[2,1:6]
    }
    return(pred_res)
    }

In [28]:
predres_mul=function(pred_matrix,data,opt_pr_mul,lengthmatrix,h,testindex)
    {
    data_length=length(testindex)
    pred_res=matrix(0,data_length,6)
    model=ets(data[[1]]$x)
    fore_l=forecast(model,h=h)
    freq=frequency(data[[1]]$x)
    for (i in 1:data_length){
    index=testindex[i]
    loc=min(opt_pr_mul[index,])
    start=lengthmatrix[i,loc]
    y_all=data[[index]]$x
    y=y_all[start:length(y_all)]
    fore_l$x=ts(y,frequency=freq,end=end(y_all))
    real=data[[index]]$xx
    fore_l$mean=ts(pred_matrix[i,],start=start(real),frequency=freq)
    res=accuracy(fore_l,real)
    pred_res[i,]=res[2,1:6]
    }
    return(pred_res)
    }

In [29]:
etsxgbcls_min=meanpre(ets_predh,ets_pr_min[,1],h,testindex)
etsxgbreg_min=meanpre(ets_predh,ets_pr_min[,2],h,testindex)
etslgbcls_min=meanpre(ets_predh,ets_pr_min[,3],h,testindex)
etslgbreg_min=meanpre(ets_predh,ets_pr_min[,4],h,testindex)

In [30]:
etsxgbcls_mean=meanpre(ets_predh,ets_pr_mean[,1],h,testindex)
etsxgbreg_mean=meanpre(ets_predh,ets_pr_mean[,2],h,testindex)
etslgbcls_mean=meanpre(ets_predh,ets_pr_mean[,3],h,testindex)
etslgbreg_mean=meanpre(ets_predh,ets_pr_mean[,4],h,testindex)

### The error of each improved prediction result

In [31]:
lengthtest=length5[testindex,]

In [32]:
etsminxgbclsres=predres(etsxgbcls_min,data,ets_pr_min[,1],lengthtest,h,testindex)
etsminxgbregres=predres(etsxgbreg_min,data,ets_pr_min[,2],lengthtest,h,testindex)
etsminlgbclsres=predres(etslgbcls_min,data,ets_pr_min[,3],lengthtest,h,testindex)
etsminlgbregres=predres(etslgbreg_min,data,ets_pr_min[,4],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

In [33]:
etsmeanxgbclsres=predres(etsxgbcls_mean,data,ets_pr_mean[,1],lengthtest,h,testindex)
etsmeanxgbregres=predres(etsxgbreg_mean,data,ets_pr_mean[,2],lengthtest,h,testindex)
etsmeanlgbclsres=predres(etslgbcls_mean,data,ets_pr_mean[,3],lengthtest,h,testindex)
etsmeanlgbregres=predres(etslgbreg_mean,data,ets_pr_mean[,4],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

In [34]:
end_time = Sys.time()
time_matrix[2,]=end_time-start_time

In [47]:
end_time-start_time

Time difference of 1.222776 mins

### The results of the two methods of determining labels are averaged respectively

In [35]:
etsallmin=(etsxgbcls_min+etslgbcls_min+etsxgbreg_min+etslgbreg_min)/4
etsallmean=(etsxgbcls_mean+etslgbcls_mean+etsxgbreg_mean+etslgbreg_mean)/4

In [36]:
etsallminres=predres_mul(etsallmin,data,ets_pr_min,lengthtest,h,testindex)
etsallmeanres=predres_mul(etsallmean,data,ets_pr_mean,lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

### Under the above classification, the improved forecast results of the classification and regression models are averaged respectively

In [37]:
etsclsmin=(etsxgbcls_min+etslgbcls_min)/2
etsregmin=(etsxgbreg_min+etslgbreg_min)/2
etsclsmean=(etsxgbcls_mean+etslgbcls_mean)/2
etsregmean=(etsxgbreg_mean+etslgbreg_mean)/2

In [38]:
etsclsminres=predres_mul(etsclsmin,data,ets_pr_min[,c(1,3)],lengthtest,h,testindex)
etsregminres=predres_mul(etsregmin,data,ets_pr_min[,c(2,4)],lengthtest,h,testindex)
etsclsmeanres=predres_mul(etsclsmean,data,ets_pr_mean[,c(1,3)],lengthtest,h,testindex)
etsregmeanres=predres_mul(etsregmean,data,ets_pr_mean[,c(2,4)],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

### Random slice

In [39]:
etspred_change<- aperm(ets_predh[1:datalength,1:m,1:n,1:h], c(1, 3, 2, 4))  
# 使用array函数将其转换为新的形状
etspred_change <- array(etspred_change, dim = c(datalength, m*n, h))

In [40]:
model=ets(data[[1]]$x)
fore_l=forecast(model,h=h)
etsrandomres=matrix(0,datalength,6)
for (i in 1:datalength){
    random=sample(m*n,n)
    minloc=min(random)
    # 使用cut函数将数据分割为区间
    loc <- cut(minloc, breaks =seq(0,m*n,n), labels = FALSE)
    loc<-as.numeric(loc)
    etsrandompred=apply(etspred_change[i,random,],2,mean)
    start=length5[i,loc]
    y_all=data[[i]]$x
    y=y_all[start:length(y_all)]
    fore_l$x=ts(y,frequency=freq,end=end(y_all))
    real=data[[i]]$xx
    fore_l$mean=ts(etsrandompred,start=start(real),frequency=freq)
    res=accuracy(fore_l,real)
    etsrandomres[i,]=res[2,1:6]
    }

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

In [41]:
mean1=function(x)
    {
# 使用is.finite函数检查哪些值是有限的
finite_values <- x[is.finite(x)]
# 计算有限值的平均值
d=mean(finite_values, na.rm = TRUE)
return(d)
}

In [42]:
print(apply(ets_pred_res[testindex,1,1,],2,mean1))
print(apply(etsminxgbclsres,2,mean1))
print(apply(etsminxgbregres,2,mean1))
print(apply(etsminlgbclsres,2,mean1))
print(apply(etsminlgbregres,2,mean1))

[1] -191.892614  388.394058  317.357259  -14.834792   28.066622    1.795074
[1] 338.153627 871.922133 731.310030  -7.913682  28.294438   2.268501
[1] -82.016704 510.159873 420.538506  -9.415463  27.899972   1.934111
[1] -13.485069 580.230286 482.330673 -10.532518  28.486240   1.927995
[1] -176.971940  416.253789  338.283670  -11.649845   28.132469    1.943472


In [43]:
print(apply(etsmeanxgbclsres,2,mean1))
print(apply(etsmeanxgbregres,2,mean1))
print(apply(etsmeanlgbclsres,2,mean1))
print(apply(etsmeanlgbregres,2,mean1))

[1] -199.768110  418.451343  337.808481  -12.616706   29.015860    1.969084
[1] -205.072493  423.030758  342.806042  -13.105945   28.024776    1.843602
[1]  601.315330 1062.572388  901.922824   -6.000994   28.088388    2.256251
[1] -196.647206  410.875212  334.991578  -12.875704   28.085435    1.914326


In [44]:
print(apply(etsallminres,2,mean1))
print(apply(etsallmeanres,2,mean1))

[1]  16.419979 514.494532 429.825178  -9.877877  27.682190   1.942985
[1]  -0.0431199 455.8786974 375.5887065 -11.1498372  27.8320640   1.9181758


In [45]:
print(apply(etsclsminres,2,mean1))
print(apply(etsregminres,2,mean1))
print(apply(etsclsmeanres,2,mean1))
print(apply(etsregmeanres,2,mean1))

[1] 162.334279 659.326953 554.413895  -9.223100  27.969088   2.019679
[1] -129.494322  452.817267  373.404756  -10.532654   27.872821    1.924312
[1] 200.773610 648.594581 543.506350  -9.308850  28.148951   2.029911
[1] -200.859849  416.610952  338.738813  -12.990824   27.920466    1.857166


In [46]:
print(apply(etsrandomres[testindex,],2,mean))

[1] 240.146128 773.572841 645.606013 -10.996348  29.010695   2.012234


In [47]:
etsall=matrix(0,16,6)
etsall[1,]=apply(ets_pred_res[testindex,1,1,],2,mean1)[1:6]
etsall[2,]=apply(etsminxgbclsres,2,mean1)
etsall[3,]=apply(etsminxgbregres,2,mean1)
etsall[4,]=apply(etsminlgbclsres,2,mean1)
etsall[5,]=apply(etsminlgbregres,2,mean1)
etsall[6,]=apply(etsmeanxgbclsres,2,mean1)
etsall[7,]=apply(etsmeanxgbregres,2,mean1)
etsall[8,]=apply(etsmeanlgbclsres,2,mean1)
etsall[9,]=apply(etsmeanlgbregres,2,mean1)
etsall[10,]=apply(etsallminres,2,mean1)
etsall[11,]=apply(etsallmeanres,2,mean1)
etsall[12,]=apply(etsclsminres,2,mean1)
etsall[13,]=apply(etsregminres,2,mean1)
etsall[14,]=apply(etsclsmeanres,2,mean1)
etsall[15,]=apply(etsregmeanres,2,mean1)
etsall[16,]=apply(etsrandomres[testindex,],2,mean)

In [48]:
etsall

-191.89261,388.3941,317.3573,-14.834792,28.06662,1.795074
228.19966,749.3995,624.2218,-7.905107,27.14226,2.112637
-193.86076,414.4066,335.2704,-11.913750,27.31991,1.809908
-10.32681,576.7093,478.7650,-10.503769,28.48960,1.930483
-189.62421,407.1344,329.5899,-11.812164,27.93070,1.941372
-205.98576,422.9307,344.2584,-12.707141,28.79292,1.942253
-201.69271,422.3271,342.7892,-12.473361,28.60978,1.974026
163.99351,689.3139,581.6390,-6.689387,27.48146,2.122470
-196.92332,411.8335,336.7437,-12.811389,28.17201,1.918292
-41.40303,457.1410,378.3695,-10.533697,27.20912,1.849526
-110.15207,448.3650,371.6000,-11.170320,27.80251,1.917348


In [49]:
write.csv(etsall,'ets_Hourly_final_res.csv')

## thetaf

### The error of each improved prediction result

In [48]:
start_time = Sys.time()

In [49]:
thetafxgbcls_min=meanpre(thetaf_predh,thetaf_pr_min[,1],h,testindex)
thetafxgbreg_min=meanpre(thetaf_predh,thetaf_pr_min[,2],h,testindex)
thetaflgbcls_min=meanpre(thetaf_predh,thetaf_pr_min[,3],h,testindex)
thetaflgbreg_min=meanpre(thetaf_predh,thetaf_pr_min[,4],h,testindex)

In [50]:
thetafxgbcls_mean=meanpre(thetaf_predh,thetaf_pr_mean[,1],h,testindex)
thetafxgbreg_mean=meanpre(thetaf_predh,thetaf_pr_mean[,2],h,testindex)
thetaflgbcls_mean=meanpre(thetaf_predh,thetaf_pr_mean[,3],h,testindex)
thetaflgbreg_mean=meanpre(thetaf_predh,thetaf_pr_mean[,4],h,testindex)

In [51]:
thetafminxgbclsres=predres(thetafxgbcls_min,data,thetaf_pr_min[,1],lengthtest,h,testindex)
thetafminxgbregres=predres(thetafxgbreg_min,data,thetaf_pr_min[,2],lengthtest,h,testindex)
thetafminlgbclsres=predres(thetaflgbcls_min,data,thetaf_pr_min[,3],lengthtest,h,testindex)
thetafminlgbregres=predres(thetaflgbreg_min,data,thetaf_pr_min[,4],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

In [52]:
thetafmeanxgbclsres=predres(thetafxgbcls_mean,data,thetaf_pr_mean[,1],lengthtest,h,testindex)
thetafmeanxgbregres=predres(thetafxgbreg_mean,data,thetaf_pr_mean[,2],lengthtest,h,testindex)
thetafmeanlgbclsres=predres(thetaflgbcls_mean,data,thetaf_pr_mean[,3],lengthtest,h,testindex)
thetafmeanlgbregres=predres(thetaflgbreg_mean,data,thetaf_pr_mean[,4],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

In [53]:
end_time = Sys.time()
time_matrix[3,]=end_time-start_time

In [54]:
end_time-start_time

Time difference of 48.77701 secs

### The results of the two methods of determining labels are averaged respectively

In [55]:
thetafallmin=(thetafxgbcls_min+thetaflgbcls_min+thetafxgbreg_min+thetaflgbreg_min)/4
thetafallmean=(thetafxgbcls_mean+thetaflgbcls_mean+thetafxgbreg_mean+thetaflgbreg_mean)/4

In [56]:
thetafallminres=predres_mul(thetafallmin,data,thetaf_pr_min,lengthtest,h,testindex)
thetafallmeanres=predres_mul(thetafallmean,data,thetaf_pr_mean,lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

### Under the above classification, the improved forecast results of the classification and regression models are averaged respectively

In [57]:
thetafclsmin=(thetafxgbcls_min+thetaflgbcls_min)/2
thetafregmin=(thetafxgbreg_min+thetaflgbreg_min)/2
thetafclsmean=(thetafxgbcls_mean+thetaflgbcls_mean)/2
thetafregmean=(thetafxgbreg_mean+thetaflgbreg_mean)/2

In [58]:
thetafclsminres=predres_mul(thetafclsmin,data,thetaf_pr_min[,c(1,3)],lengthtest,h,testindex)
thetafregminres=predres_mul(thetafregmin,data,thetaf_pr_min[,c(2,4)],lengthtest,h,testindex)
thetafclsmeanres=predres_mul(thetafclsmean,data,thetaf_pr_mean[,c(1,3)],lengthtest,h,testindex)
thetafregmeanres=predres_mul(thetafregmean,data,thetaf_pr_mean[,c(2,4)],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

### Random slice

In [59]:
thetafpred_change<- aperm(thetaf_predh[1:datalength,1:m,1:n,1:h], c(1, 3, 2, 4))  
# 使用array函数将其转换为新的形状
thetafpred_change <- array(thetafpred_change, dim = c(datalength, m*n, h))

In [60]:
model=ets(data[[1]]$x)
fore_l=forecast(model,h=h)
thetafrandomres=matrix(0,datalength,6)
for (i in 1:datalength){
    random=sample(m*n,n)
    minloc=min(random)
    # 使用cut函数将数据分割为区间
    loc <- cut(minloc, breaks =seq(0,m*n,n), labels = FALSE)
    loc<-as.numeric(loc)
    thetafrandompred=apply(thetafpred_change[i,random,],2,mean)
    start=length5[i,loc]
    y_all=data[[i]]$x
    y=y_all[start:length(y_all)]
    fore_l$x=ts(y,frequency=freq,end=end(y_all))
    real=data[[i]]$xx
    fore_l$mean=ts(thetafrandompred,start=start(real),frequency=freq)
    res=accuracy(fore_l,real)
    thetafrandomres[i,]=res[2,1:6]
    }

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

In [61]:
print(apply(thetaf_pred_res[testindex,1,1,],2,mean1))
print(apply(thetafminxgbclsres,2,mean1))
print(apply(thetafminxgbregres,2,mean1))
print(apply(thetafminlgbclsres,2,mean1))
print(apply(thetafminlgbregres,2,mean1))

[1] -169.292991  393.940515  320.362836   -7.097676   23.899342    2.496652
[1] -191.799005  455.172480  370.728594   -4.574842   28.575591    3.733098
[1] -185.150907  432.491948  350.852398   -4.953046   25.207061    3.164992
[1] -178.195664  412.459509  331.527688   -7.669950   25.742149    2.512162
[1] -162.437009  417.344729  334.869155   -6.821734   27.469947    3.500030


In [62]:
print(apply(thetafmeanxgbclsres,2,mean1))
print(apply(thetafmeanxgbregres,2,mean1))
print(apply(thetafmeanlgbclsres,2,mean1))
print(apply(thetafmeanlgbregres,2,mean1))

[1] -156.984741  394.936357  316.560698   -3.969177   23.861095    1.682101
[1] -173.372232  402.148154  323.228261   -7.242272   24.198349    1.728107
[1] -205.298637  470.242176  381.381989   -4.215325   26.658590    2.618807
[1] -174.734207  403.985087  324.683959   -6.869637   23.815871    1.681491


In [63]:
print(apply(thetafallminres,2,mean1))
print(apply(thetafallmeanres,2,mean1))

[1] -179.395646  422.486886  343.793032   -6.004893   26.121875    2.749248
[1] -177.597454  410.149023  331.368516   -5.574103   24.209618    1.783868


In [64]:
print(apply(thetafallminres,2,mean1))
print(apply(thetafallmeanres,2,mean1))

[1] -179.395646  422.486886  343.793032   -6.004893   26.121875    2.749248
[1] -177.597454  410.149023  331.368516   -5.574103   24.209618    1.783868


In [65]:
print(apply(thetafclsminres,2,mean1))
print(apply(thetafclsmeanres,2,mean1))
print(apply(thetafregminres,2,mean1))
print(apply(thetafregmeanres,2,mean1))

[1] -184.997334  427.775440  348.830462   -6.122396   26.502876    2.529410
[1] -181.141689  424.356986  344.448993   -4.092251   24.879101    1.991001
[1] -173.793958  421.024467  341.932849   -5.887390   26.229511    3.253258
[1] -174.053219  402.913137  323.875879   -7.055955   23.997987    1.698361


In [66]:
print(apply(thetafrandomres[testindex,],2,mean1))

[1] -225.956033  486.534949  398.392340   -7.252917   26.544386    2.246822


In [69]:
thetafall=matrix(0,16,6)
thetafall[1,]=apply(thetaf_pred_res[testindex,1,1,],2,mean1)[1:6]
thetafall[2,]=apply(thetafminxgbclsres,2,mean1)
thetafall[3,]=apply(thetafminxgbregres,2,mean1)
thetafall[4,]=apply(thetafminlgbclsres,2,mean1)
thetafall[5,]=apply(thetafminlgbregres,2,mean1)
thetafall[6,]=apply(thetafmeanxgbclsres,2,mean1)
thetafall[7,]=apply(thetafmeanxgbregres,2,mean1)
thetafall[8,]=apply(thetafmeanlgbclsres,2,mean1)
thetafall[9,]=apply(thetafmeanlgbregres,2,mean1)
thetafall[10,]=apply(thetafallminres,2,mean1)
thetafall[11,]=apply(thetafallmeanres,2,mean1)
thetafall[12,]=apply(thetafclsminres,2,mean1)
thetafall[13,]=apply(thetafclsmeanres,2,mean1)
thetafall[14,]=apply(thetafregminres,2,mean1)
thetafall[15,]=apply(thetafregmeanres,2,mean1)
thetafall[16,]=apply(thetafrandomres[testindex,],2,mean1)

In [70]:
thetafall

-169.2930,393.9405,320.3628,-7.097676,23.89934,2.496652
-183.7602,443.9041,362.0356,-2.434450,27.25387,4.008915
-187.0906,431.3963,350.8069,-5.057846,24.55066,2.350386
-173.2831,412.3020,330.8763,-6.551747,26.66984,2.662040
-159.8001,412.7041,330.7016,-6.175379,25.87798,3.982469
-163.1182,391.6258,314.6899,-5.369127,22.60483,1.647802
-171.1062,400.5604,323.4094,-6.849556,23.65010,1.799985
-204.0081,469.4364,379.8645,-4.888662,26.29946,2.520095
-173.9951,402.0373,323.6405,-6.829724,23.74565,1.697881
-175.9835,418.2761,340.4447,-5.054855,25.42090,2.659950
-178.0569,408.6053,330.5912,-5.984267,23.69116,1.772341


In [71]:
write.csv(thetafall,'thetaf_Hourly_final_res.csv')

In [72]:
time_matrix

user_time,system_time,elapsed_time
14.11554,14.11554,14.11554
52.52639,52.52639,52.52639
51.63845,51.63845,51.63845


In [73]:
write.csv(time_matrix,'Hourly_time_matrix.csv')